# Trabalho Prático 01 - Iluminação (Flat, Gouraud, Phong)

- Thiago Martin Poppe
- 2017014324

# Breve discussão sobre os shaders

## Flat Shading

- Em um modelo de iluminação do tipo Flat Shading, aplicaremos o modelo de Phong sobre as normais de cada face do nosso objeto. Para obtermos as normais das faces, usamos a função gluQuadricNormals passando como parâmetro a quádrica e GL_FLAT. Isso faz com que a quádrica gere normais apenas nas faces. Juntamente, nos shaders, utilizamos a "key-word" flat, que indica que não queremos que a OpenGL faça a interpolação dos valores obtidos.
<br><br>
- Obs: Para a Utah Teapot não tivemos o artifício de utilizar o gluQuadricNormals para definir as normais da forma que queriamos. Então, creio que o formato de iluminação gerado utilizou as normais dos vértices, que aparenta ser o padrão gerado pela glutSolidTeapot, ao invés da normal das faces.

## Gouraud Shading

- Em um modelo de iluminação do tipo Gouraud Shading, aplicaremos o modelo de Phong sobre as normais de cada vértice do nosso objeto. Para obtermos as normais dos vértices, usamos a função gluQuadricNormals passando como parâmetro a quádrica e GL_SMOOTH. Isso faz com que a quádrica gere normais apenas nos vértices (que é o valor default definido pela OpenGL para essa função). Quanto aos shaders, utilizamos o mesmo shader usado no Flat Shading, porém, as normais passadas para os mesmos serão as normais dos vértices, como dito anteriormente.
<br><br>
- Para o componente especular, temos que o "highlight" está mais bem definido. Porém, ainda temos uma aparência de "borrão" nele. Veremos que isso ficará mais bem definido usando o próximo modelo de iluminação.

## Phong Shading

- Em um modelo de iluminação do tipo Phong Shading, aplicaremos o modelo de Phong sobre as normais de cada vértice interpoladas do nosso objeto. Em outras palavras, iremos aplicar o modelo "pixel a pixel". Para obtermos as normais dos vértices, usamos o mesmo procedimento anterior, e a OpenGL realiza o processo de interpolação dos valores que precisamos. Quanto aos shaders, utilizamos o mesmo shader usado no Gouraud Shading, porém, o cálculo da intensidade da luz se dará no fragment shader, pois faremos o mesmo "pixel a pixel", como dito anteriormente. Com isso, temos que o "highlight" se torna mais bem definido, e com uma aparência mais voltada para o que realmente vemos.
<br><br>
- Podemos ver a mudança do "highlight" bem pronunciada na esfera, onde no Phong Shading temos um círculo com mais iluminação do que o restante da esfera.

## Observações

- Obs1.: Não conseguimos aplicar nenhum shader para o gluCylinder. Não sei descobri o motivo pelo qual não foi possível. Tentei utilizar algum outro objeto do glut, mas todos estavam em uma posição desfavorável e quando tentava aplicar transformações de rotação toda a cena rotacionava ao invés de apenas o objeto. Logo, optei por deixar apenas os shaders aplicados sobre a esfera e o Utah Teapot, que já apresentaram resultados interessantes.
<br><br>
- Obs2.: Afim de utilizar o glutSolidTeapot, tivemos que iniciar o glut e criar uma janela utilizando o glut. Tal processo foi desnecessário, mas sem o mesmo a Utah Teapot gerava erro. Por algum motivo, a janela do glut ao ser fechada acaba resultando em "dead kernel". Para evitar tal problema, ao observar os resultados obtidos, sugiro executar todas as células sem fechar as janelas extras geradas pelo glut.
<br><br>
- Obs3.: Utilizo a biblioteca glfw para criar janelas e realizar o loop principal.
<br><br>
- Obs4.: Utilizamos a versão 330 compatibility nos shaders afim de poder utilizar as variáveis gl_Position, gl_Normal, etc. Com isso, conseguimos obter e definir a posição dos vértices e normais do nosso objeto, bem como ter acesso às matrizes de ModelView e Perspective.

# Imports

In [1]:
import sys
import glfw
import OpenGL.GL.shaders as gl_shaders

from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *

# Definindo variáveis globais

In [2]:
width, height = 800, 600

# Definindo shaders para Flat Shading

In [3]:
# Definindo o vertex_shader
vertex_shader_flat = """
#version 330 compatibility

// Uniform para a posição da fonte luz
uniform vec3 lightPosition;

// Váriaveis const float para guardar a contribuição especular e difusa
const float specularContrib = 0.4;
const float diffuseContrib = 1.0 - specularContrib;

// Váriavel para guardar a intensidade da luz naquele ponto
// O identificador flat indica que não iremos interpolar os valores
flat out float intensity;

void main() {
    // Vec3 para a posição do nosso vértice e normal
    vec3 p = vec3(gl_ModelViewMatrix * gl_Vertex);
    vec3 n = normalize(gl_NormalMatrix * gl_Normal);
    
    // Computando a direção da luz
    vec3 lightDir = normalize(lightPosition - p);
    
    // Computando o vetor R
    vec3 R = reflect(lightDir, n);
    
    // Computando o vetor viewVec
    vec3 viewVec = normalize(-p);
    
    // Computando o componente difuso (produto escalar entre a direção da luz e a normal)
    float diffuse = max(0.0, dot(lightDir, n));
    
    // Computando o componente especular
    float spec = 0.0;
    if (diffuse > 0.0) {
        // Cosseno do ângulo entre R e viewVec (em outras palavras, produto escalar)
        spec = max(0.0, dot(R, viewVec));
        
        // Parte da fórmula do cosseno elevado a um "n"
        spec = pow(spec, 64.0);
    }
    
    // Calculando a intensidade da luz
    intensity = (diffuse * diffuseContrib) + (spec * specularContrib);
    
    // Aplicando a matriz de ModelView e Projection sobre o nosso vértice de entrada
    gl_Position = ftransform();
}
"""

# Definindo o fragment_shader
fragment_shader_flat = """
#version 330 compatibility

// Uniform para a cor do nosso objeto
uniform vec3 objColor;

// Váriavel para guardar a intensidade da luz naquele ponto
// O identificador flat indica que não iremos interpolar os valores
flat in float intensity;

// Cor de saída
out vec4 outColor;

void main() {
    // Definindo uma luz ambiente
    vec3 ambientLight = vec3(0.15, 0.1, 0.1);
    
    // Exibindo a cor
    outColor = vec4(ambientLight + objColor * intensity, 1.0);
}
"""

# Definindo shaders para o Gouraud Shading

In [4]:
# Criando o vertex_shader
vertex_shader_gouraud = """
#version 330 compatibility

// Uniform para a posição da fonte luz
uniform vec3 lightPosition;

// Váriaveis const float para guardar a contribuição especular e difusa
const float specularContrib = 0.4;
const float diffuseContrib = 1.0 - specularContrib;

// Váriavel para guardar a intensidade da luz naquele ponto
// Note que dessa vez iremos interpolar os valores
varying float intensity;

void main() {
    // Vec3 para a posição do nosso vértice e normal
    vec3 p = vec3(gl_ModelViewMatrix * gl_Vertex);
    vec3 n = normalize(gl_NormalMatrix * gl_Normal);
    
    // Computando a direção da luz
    vec3 lightDir = normalize(lightPosition - p);
    
    // Computando o vetor R
    vec3 R = reflect(lightDir, n);

    // Computando o vetor viewVec
    vec3 viewVec = normalize(-p);
    
    // Computando o componente difuso (produto escalar entre a direção da luz e a normal)
    float diffuse = max(0.0, dot(lightDir, n));
    
    // Computando o componente especular
    float spec = 0.0;
    if (diffuse > 0.0) {
        // Cosseno do ângulo entre R e viewVec (em outras palavras, produto escalar)
        spec = max(0.0, dot(R, viewVec));
        
        // Parte da fórmula do cosseno elevado a um "n"
        spec = pow(spec, 64.0);
    }

    // Calculando a intensidade da luz
    intensity = (diffuse * diffuseContrib) + (spec * specularContrib);
    
    // Aplicando a matriz de ModelView e Projection sobre o nosso vértice de entrada
    gl_Position = ftransform();
}
"""

# Criando o fragment_shader
fragment_shader_gouraud = """
#version 330 compatibility

// Uniform para a cor do nosso objeto
uniform vec3 objColor;

// Váriavel para guardar a intensidade da luz naquele ponto
// Note que dessa vez iremos interpolar os valores
varying float intensity;

// Cor de saída
out vec4 outColor;

void main() {
    // Definindo uma luz ambiente
    vec3 ambientLight = vec3(0.15, 0.1, 0.1);
    
    // Exibindo a cor
    outColor = vec4(ambientLight + objColor * intensity, 1.0);
}
"""

# Definindo shaders para o Phong Shading

In [5]:
# Criando o vertex_shader
vertex_shader_phong = """
#version 330 compatibility

// Iremos passar para o fragment shader as posições dos vértices e as suas normais
out vec4 position;
out vec3 normal;

void main() {
    // Computando a normal (aplicando a transposta da inversa)
    normal = normalize(gl_NormalMatrix * gl_Normal);
    
    // Computando as posições dos vértices (aplicando a matriz de ModelView)
    position = gl_ModelViewMatrix * gl_Vertex;
    
    // Aplicando a matriz de ModelView e Projection sobre o nosso vértice de entrada
    gl_Position = ftransform();
}
"""

# Criando o fragment_shader
fragment_shader_phong = """
#version 330 compatibility

// Recebemos do vertex shader os valores das posições dos vértices e as suas normais
in vec4 position;
in vec3 normal;

// Uniform para a posição da fonte luz
uniform vec3 lightPosition;

// Uniform para a cor do nosso objeto
uniform vec3 objColor;

// Váriaveis const float para guardar a contribuição especular e difusa
const float specularContrib = 0.4;
const float diffuseContrib = 1.0 - specularContrib;

// Cor de saída
out vec4 outColor;

void main() {
    // Vec3 para a posição do nosso vértice e normal
    vec3 p = vec3(gl_ModelViewMatrix * position);
    vec3 n = normalize(gl_NormalMatrix * normal);

    // Computando a direção da nossa luz
    vec3 lightDir = normalize(lightPosition - p);
    
    // Computando o R
    vec3 R = reflect(lightDir, n);
    
    // Computando o viewVec
    vec3 viewVec = normalize(-p);
    
    // Computando o componente difuso (produto escalar entre a direção da luz e a normal)
    float diffuse = max(0.0, dot(lightDir, n));
    
    // Computando o componente especular
    float spec = 0.0;
    if (diffuse > 0.0) {
        // Cosseno do ângulo entre R e viewVec (em outras palavras, produto escalar)
        spec = max(0.0, dot(R, viewVec));
        
        // Parte da fórmula do cosseno elevado a um "n"
        spec = pow(spec, 64.0);
    }

    // Calculando a intensidade da luz
    float intensity = (diffuse * diffuseContrib) + (spec * specularContrib);
    
    // Definindo uma luz de ambiente
    vec3 ambientLight = vec3(0.15, 0.1, 0.1);
    
    // Exibindo a cor
    outColor = vec4(ambientLight + objColor * intensity, 1.0);
}
"""

# Rode essa célula para gerar uma esfera usando Flat Shading

In [6]:
# Inicializando o glfw
if not glfw.init():
    print('Não foi possível iniciar o glfw')
    exit()

# Criando uma janela
window = glfw.create_window(width, height, 'Flat Shading Sphere', None, None)
if not window:
    print('Não foi possível criar a janela')
    glfw.terminate()
    exit()

# Tornando a janela o contexto atual
glfw.make_context_current(window)

# Compilando e usando o shader
shader = gl_shaders.compileProgram(gl_shaders.compileShader(vertex_shader_flat, GL_VERTEX_SHADER),
                                   gl_shaders.compileShader(fragment_shader_flat, GL_FRAGMENT_SHADER))
glUseProgram(shader)

# Definindo a cor de limpeza da tela
glClearColor(0.2, 0.2, 0.2, 1.0)

# Habilitando o DEPTH_TEST
glEnable(GL_DEPTH_TEST)

# Definindo a posição da nossa luz
lightPositionLoc = glGetUniformLocation(shader, 'lightPosition')
glUniform3f(lightPositionLoc, 0.0, 1.0, 2.0)

# Definindo a cor do nosso objeto
objColorLoc = glGetUniformLocation(shader, 'objColor')
glUniform3f(objColorLoc, 1.0, 0.0, 0.0)

# Definindo o tipo de projeção
glMatrixMode(GL_PROJECTION)
glLoadIdentity()
gluPerspective(45.0, width/height, 0.1, 100.0)

# Transladando a esfera para aparecer na tela
glTranslatef(0.0, 0.0, -5.0)

# Criando uma quádrica para a esfera e definindo as propriedades da mesma
quad = gluNewQuadric()
gluQuadricNormals(quad, GLU_FLAT)

# MAIN LOOP
while not glfw.window_should_close(window):
    # Capturando eventos
    glfw.poll_events()

    # Limpando os buffers
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    # Gerando uma esfera usando glu
    gluSphere(quad, 1, 30, 30)

    # Desenhando na tela
    glfw.swap_buffers(window)

# Terminando o glfw
glfw.terminate()

# Rode essa célula para gerar uma esfera usando Gouraud Shading

In [7]:
# Inicializando o glfw
if not glfw.init():
    print('Não foi possível inicializar o glfw')
    exit()

# Criando uma janela
window = glfw.create_window(width, height, 'Gouraud Shading Sphere', None, None)
if not window:
    print('Não foi possível criar a janela')
    glfw.terminate()
    exit()

# Tornando a janela criada o contexto atual
glfw.make_context_current(window)

# Compilando e usando o shader
shader = gl_shaders.compileProgram(gl_shaders.compileShader(vertex_shader_gouraud, GL_VERTEX_SHADER),
                                   gl_shaders.compileShader(fragment_shader_gouraud, GL_FRAGMENT_SHADER))
glUseProgram(shader)

# Definindo a cor de limpeza da tela
glClearColor(0.2, 0.2, 0.2, 1.0)

# Habilitando o DEPTH_TEST
glEnable(GL_DEPTH_TEST)

# Definindo a posição da nossa luz
lightPositionLoc = glGetUniformLocation(shader, 'lightPosition')
glUniform3f(lightPositionLoc, 0.0, 1.0, 2.0)

# Definindo a cor do nosso objeto
objColorLoc = glGetUniformLocation(shader, 'objColor')
glUniform3f(objColorLoc, 1.0, 0.0, 0.0)

# Definindo o tipo de projeção
glMatrixMode(GL_PROJECTION)
glLoadIdentity()
gluPerspective(45.0, width/height, 0.1, 100.0)

# Transladando a esfera para aparecer na tela
glTranslatef(0.0, 0.0, -5.0)

# Criando uma quádrica para a esfera e definindo as propriedades da mesma
quadSphere = gluNewQuadric()
gluQuadricNormals(quadSphere, GL_SMOOTH)

# MAIN LOOP
while not glfw.window_should_close(window):
    # Capturando eventos
    glfw.poll_events()

    # Limpando os buffers
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    # Gerando uma esfera usando glu
    gluSphere(quadSphere, 1, 30, 30)

    # Desenhando na tela
    glfw.swap_buffers(window)

# Terminando o glfw
glfw.terminate()

# Rode essa célula para gerar uma esfera usando Phong Shading

In [8]:
# Inicializando o glfw
if not glfw.init():
    print('Não foi possível inicializar o glfw')
    exit()

# Criando uma janela
window = glfw.create_window(width, height, 'Phong Shading Sphere', None, None)
if not window:
    print('Não foi possível criar a janela')
    glfw.terminate()
    exit()

# Tornando a janela criada o contexto atual
glfw.make_context_current(window)

# Compilando e usando o shader
shader = gl_shaders.compileProgram(gl_shaders.compileShader(vertex_shader_phong, GL_VERTEX_SHADER),
                                   gl_shaders.compileShader(fragment_shader_phong, GL_FRAGMENT_SHADER))
glUseProgram(shader)

# Definindo a cor de limpeza da tela
glClearColor(0.2, 0.2, 0.2, 1.0)

# Habilitando o DEPTH_TEST
glEnable(GL_DEPTH_TEST)

# Definindo a posição da nossa luz
lightPositionLoc = glGetUniformLocation(shader, 'lightPosition')
glUniform3f(lightPositionLoc, 0.0, 1.0, 2.0)

# Definindo a cor do nosso objeto
objColorLoc = glGetUniformLocation(shader, 'objColor')
glUniform3f(objColorLoc, 1.0, 0.0, 0.0)

# Definindo o tipo de projeção
glMatrixMode(GL_PROJECTION)
glLoadIdentity()
gluPerspective(45.0, width/height, 0.1, 100.0)

# Transladando a esfera para aparecer na tela

glTranslatef(0.0, 0.0, -5.0)

# Criando uma quádrica para a esfera e definindo as propriedades da mesma
quadSphere = gluNewQuadric()
gluQuadricNormals(quadSphere, GL_SMOOTH)

# MAIN LOOP
while not glfw.window_should_close(window):
    # Capturando eventos
    glfw.poll_events()

    # Limpando os buffers
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    # Gerando uma esfera usando glu
    gluSphere(quadSphere, 1, 100, 100)

    # Desenhando na tela
    glfw.swap_buffers(window)

# Terminando o glfw
glfw.terminate()

# Rode essa célula para gerar uma Utah Teapot usando Flat Shading

In [9]:
# Inicializando o glut
glutInit(sys.argv)
glutCreateWindow('Unnecessary Window')

# Inicializando o glfw
if not glfw.init():
    print('Não foi possível inicializar o glfw')
    exit()

# Criando uma janela
window = glfw.create_window(width, height, 'Flat Shading Utah Teapot', None, None)
if not window:
    print('Não foi possível criar a janela')
    glfw.terminate()
    exit()

# Tornando a janela criada o contexto atual
glfw.make_context_current(window)

# Compilando e usando o shader
shader = gl_shaders.compileProgram(gl_shaders.compileShader(vertex_shader_flat, GL_VERTEX_SHADER),
                                   gl_shaders.compileShader(fragment_shader_flat, GL_FRAGMENT_SHADER))
glUseProgram(shader)

# Definindo a cor de limpeza da tela
glClearColor(0.2, 0.2, 0.2, 1.0)

# Habilitando o DEPTH_TEST
glEnable(GL_DEPTH_TEST)

# Definindo a posição da nossa luz
lightPositionLoc = glGetUniformLocation(shader, 'lightPosition')
glUniform3f(lightPositionLoc, 0.0, 1.0, 2.0)

# Definindo a cor do nosso objeto
objColorLoc = glGetUniformLocation(shader, 'objColor')
glUniform3f(objColorLoc, 1.0, 0.0, 0.0)

# Definindo o tipo de projeção
glMatrixMode(GL_PROJECTION)
glLoadIdentity()
gluPerspective(45.0, width/height, 0.1, 100.0)

# Transladando a Utah Teapot para aparecer na tela
glTranslatef(0.0, 0.0, -5.0)

# MAIN LOOP
while not glfw.window_should_close(window):
    # Capturando eventos
    glfw.poll_events()

    # Limpando os buffers
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    # Gerando uma Utah Teapot com glut
    glutSolidTeapot(1)

    # Desenhando na tela
    glfw.swap_buffers(window)

# Terminando o glfw
glfw.terminate()

# Rode essa célula para gerar uma Utah Teapot usando Gouraud Shading

In [10]:
# Inicializando o glut
glutInit(sys.argv)
glutCreateWindow('Unnecessary Window')

# Inicializando o glfw
if not glfw.init():
    print('Não foi possível inicializar o glfw')
    exit()

# Criando uma janela
window = glfw.create_window(width, height, 'Gouraud Shading Utah Teapot', None, None)
if not window:
    print('Não foi possível criar a janela')
    glfw.terminate()
    exit()

# Tornando a janela criada o contexto atual
glfw.make_context_current(window)

# Compilando e usando o shader
shader = gl_shaders.compileProgram(gl_shaders.compileShader(vertex_shader_gouraud, GL_VERTEX_SHADER),
                                   gl_shaders.compileShader(fragment_shader_gouraud, GL_FRAGMENT_SHADER))
glUseProgram(shader)

# Definindo a cor de limpeza da tela
glClearColor(0.2, 0.2, 0.2, 1.0)

# Habilitando o DEPTH_TEST
glEnable(GL_DEPTH_TEST)

# Definindo a posição da nossa luz
lightPositionLoc = glGetUniformLocation(shader, 'lightPosition')
glUniform3f(lightPositionLoc, 0.0, 1.0, 2.0)

# Definindo a cor do nosso objeto
objColorLoc = glGetUniformLocation(shader, 'objColor')
glUniform3f(objColorLoc, 1.0, 0.0, 0.0)

# Definindo o tipo de projeção
glMatrixMode(GL_PROJECTION)
glLoadIdentity()
gluPerspective(45.0, width/height, 0.1, 100.0)

# Transladando a Utah Teapot para aparecer na tela
glTranslatef(0.0, 0.0, -5.0)

# MAIN LOOP
while not glfw.window_should_close(window):
    # Capturando eventos
    glfw.poll_events()

    # Limpando os buffers
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    # Gerando uma Utah Teapot com glut
    glutSolidTeapot(1)

    # Desenhando na tela
    glfw.swap_buffers(window)

# Terminando o glfw
glfw.terminate()

# Rode essa célula para gerar uma Utah Teapot usando Phong Shading

In [11]:
# Inicializando o glut
glutInit(sys.argv)
glutCreateWindow('Unnecessary Window')

# Inicializando o glfw
if not glfw.init():
    print('Não foi possível inicializar o glfw')
    exit()

# Criando uma janela
window = glfw.create_window(width, height, 'Phong Shading Utah Teapot', None, None)
if not window:
    print('Não foi possível criar a janela')
    glfw.terminate()
    exit()

# Tornando a janela criada o contexto atual
glfw.make_context_current(window)

# Compilando e usando o shader
shader = gl_shaders.compileProgram(gl_shaders.compileShader(vertex_shader_phong, GL_VERTEX_SHADER),
                                   gl_shaders.compileShader(fragment_shader_phong, GL_FRAGMENT_SHADER))
glUseProgram(shader)

# Definindo a cor de limpeza da tela
glClearColor(0.2, 0.2, 0.2, 1.0)

# Habilitando o DEPTH_TEST
glEnable(GL_DEPTH_TEST)

# Definindo a cor do nosso objeto
objColorLoc = glGetUniformLocation(shader, 'objColor')
glUniform3f(objColorLoc, 1.0, 0.0, 0.0)

# Definindo o tipo de projeção
glMatrixMode(GL_PROJECTION)
glLoadIdentity()
gluPerspective(45.0, width/height, 0.1, 100.0)

# Transladando a Utah Teapot para aparecer na tela
glTranslatef(0.0, 0.0, -5.0)

# MAIN LOOP
while not glfw.window_should_close(window):
    # Capturando eventos
    glfw.poll_events()

    # Limpando os buffers
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    # Gerando uma Utah Teapot usando glut
    glutSolidTeapot(1)
    
    # Definindo a posição da nossa luz
    lightPositionLoc = glGetUniformLocation(shader, 'lightPosition')
    glUniform3f(lightPositionLoc, 0.0, 1.0, 2.0)

    # Desenhando na tela
    glfw.swap_buffers(window)

# Terminando o glfw
glfw.terminate()